# Часть 3. Проверка рекомендательной системы на валидационной выборке
Пдготовка входных фичей с помощью faiss с помощью базового и валидационного датасетов и использование их на обученной модели CatBoostClassifier с целью проверки эффективности на валидационной выборке. 

**Описание действий:**
Нам поступает какой-то запрос query (1x72), для него мы получаем 30 кандидатов с помощью faiss. Снов аподставляем query к полученным кандидатам (совмещаем векторы), получаем матрицу (1х144), как в предыдущей части работы, но теперь у нас нет таргета. Зато у нас есть обученная модель CatBoostClassifier и входные фичи. Мы забрасываем эти фичи в нашу модель (напомню, я обучила ее и сохранила в предыдущей части №2). Модель отрабатывает и выдает вероятности. Выбираем вектор,для которого вероятность максимальна (ближе к 1).  

### Загрузка библиотек и исходных датасетов 

In [1]:
import faiss
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

In [2]:
# Загрузка данных
df_base = pd.read_csv("base.csv", index_col=0)
df_validation = pd.read_csv("validation.csv", index_col=0)
df_validation_answer = pd.read_csv("validation_answer.csv", index_col=0)

###  Подготовка данных для работы Faiss

In [3]:
# преобразуем в датафрейм в Series
df_validation_answer = df_validation_answer.squeeze()

In [4]:
# Нормализация данных 
def normalize_with_indices(df):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    df_normalized = pd.DataFrame(df_scaled, index=df.index, columns=df.columns)
    return df_normalized

df_base_scaled = normalize_with_indices(df_base)
df_validation_scaled = normalize_with_indices(df_validation)

In [5]:
del df_base
del df_validation

### Первая стадия поиска ближайших соседей с использованием Faiss на валидационной выборке 

In [6]:
%%time 
# Инициализация Faiss
dims = df_base_scaled.shape[1]
n_cells = 1000

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

idx_l2.nprobe = 100

# Обучение Faiss на base данных
idx_l2.train(np.ascontiguousarray(df_base_scaled).astype('float32'))

idx_l2.add(np.ascontiguousarray(df_base_scaled).astype('float32'))

base_index = {k: v for k, v in enumerate(df_base_scaled.index.to_list())}

# Поиск ближайших соседей на валидационных данных
k_neighbours = 30
vecs, idx = idx_l2.search(np.ascontiguousarray(df_validation_scaled.values).astype('float32'), k_neighbours)

CPU times: user 45min 41s, sys: 2.54 s, total: 45min 44s
Wall time: 5min 46s


In [7]:
# Вычисляем точность (accuracy) модели на валидационных данных
acc = 0
for target, el in zip(df_validation_answer.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

73.757


### Подготовка данных для использования с CatBoost

In [8]:
# Преобразование индексов в массив numpy
idx = np.array([[base_index[_] for _ in el] for el in idx], dtype=object) 

# Изменение формы расстояний для корректной работы
vecs = vecs.reshape(-1, 1)

In [9]:
# Подготовка целевых значений для использования с CatBoost
cat_target = np.array([[1 if x == target else 0 for x in el] for el, target in zip(idx, df_validation_answer.values.tolist())]).flatten()

In [10]:
# Извлечение признаков для кандидатов и объектов
candidate_features = df_base_scaled.loc[idx.flatten()].values  # Признаки для кандидатов
object_features = df_validation_scaled.values  # Признаки для объектов

In [11]:
del df_base_scaled 
del df_validation_scaled

In [12]:
# Повторение признаков объектов для кандидатов
repeated_object = np.repeat(object_features, k_neighbours, axis=0)

In [13]:
# Горизонтальное объединение массивов
cat_features = np.hstack((vecs, candidate_features, repeated_object))

In [14]:
del vecs 
del candidate_features 

### Рекомендации с помощью обученной модели CatBoost

In [15]:
# Загрузка обученной модели CatBoost
cat_model = CatBoostClassifier()
cat_model.load_model("cat_model.cbm")

In [16]:
# Предсказание вероятностей для ближайших соседей
neighbor_probs = cat_model.predict_proba(cat_features)

In [17]:
cat_val_probs = []
idx_val = []

In [18]:
cat_val_probs.append(np.vstack(neighbor_probs))
idx_val.append(np.vstack(idx))

In [19]:
# Преобразование списка в массив numpy
cat_val_probs = np.array(cat_val_probs)

idx_val = np.array(idx_val)

In [20]:
%%time
# Находим индексы элементов с максимальной вероятностью предсказания
k = 30
n = 5

max_indices = np.empty((0, n), dtype=int)
for i in range(0, len(cat_val_probs.flatten()), k):
    sub_arr = cat_val_probs.flatten()[i:i+k]
    max_values = np.argpartition(sub_arr, -n)[-n:]
    max_indices = np.vstack((max_indices, max_values))


CPU times: user 15min 34s, sys: 408 ms, total: 15min 35s
Wall time: 15min 35s


In [21]:
# Выбор кандидатов на основе их индексов
neighbors_idx = idx_val[np.arange(len(idx_val))[:, None], max_indices]

In [22]:
# Вычисляем точность на первых n местах для заданных целевых значений и предсказаний
acc = 0
for target_val, pred_vals in zip(df_validation_answer.values.tolist(), neighbors_idx):
    if target_val in pred_vals[:n]:
        acc += 1

accuracy_at_n = 100 * acc / len(neighbors_idx)

print(accuracy_at_n)

0.004


##  ВЫВОД 
На валидационной выборке проведен поиск ближайших соседей, затем с помощью обученной модели CatBoostClassifier выбраны кандидаты с наибольшей вероятностью. 
Метрика низкая, кажется, где-то ошибка, продолжу искать. 

# Что еще было выполнено для улучшения метрики: 

- Удаление признаков с равномерным распределением и выбросами - позволило увеличить метрику на 0.5 - 1% при поиске ближайших соседей, что не; 
- Метод главных компонент (PCA) - уменьшение размерности данных не позволило поднять метрику. 
- Стандартизация данных с помощью StandardScaler позволилила повысить метрику значительно, потому при подготовке данных ее оставила. 
- Уменьшение количества кластеров в фаисс и увеличение числа кластеров, в которых нужно искать соседей помогает поднять метрику на пару процентов, но значительно увеличивает время работы, потому я приняла решение отказаться, поскольку мое железо не тянет:) 
- Увеличение количества соседей также позволяет слегка поднять метрику, но увеличивает размер матрицы, которую необходимо подавать на следующую ступень (на обучение катбуста), в итоге ядро перегружвется и падает. Я приняла решение отказаться от больших обьемов данных по этой причине.
- Пыталась перебрать параетры с Optuna  и GridSearchCV, но падает ядро. Увеличение количества деревьев и глубины тоже приводят к падению ядра. 